# L2 Switch Controller

The following defines a L2 Switch controller to enable forwarding of ethernet frames to appropriate ports. We use Ryu as our OpenFlow controller. More information and documentation about Ryu here: https://ryu.readthedocs.io/en/latest/developing.html

#### A few key points:
* Ryu does not natively know or care about what you do with a packet.
* All it needs to know is the output port in which to send the packet.
* Ryu does not know about internet protocols like ARP, IP, MAC forwarding etc. It does exactly what you ask it to.
* Ryu is an OpenFlow controller which is designed to run on a seperate server and usually knows the entire topology. (Think of a cluster of network switches and Ryu as the orchestrator that knows everything)
* Therefore, every variable you define in Ryu controller is common for every Switch. 
* But OpenFlow also provides a flow table that is unique to every switch which has to be populated for every switch with match-action pairs.
* Everytime a switch receives a packet, it will first check its flow table for any matching entries. Otherwise, it simply sends the packet to the controller.
* Ryu and Mininet work only on Linux.

#### We define the boilerplate template code for the Ryu Controller first:

The following lines define the imports needed for Ryu:

In [ ]:
from ryu.base import app_manager
from ryu.controller import ofp_event
from ryu.controller.handler import CONFIG_DISPATCHER, MAIN_DISPATCHER
from ryu.controller.handler import set_ev_cls
from ryu.lib.packet import packet, ethernet
from ryu.ofproto import ofproto_v1_3

Ryu implements the OpenFlow protocol and therefore the terminologies are greatly dependent on the OpenFlow protocol itself. 

In [ ]:
class L2SwitchController(app_manager.RyuApp):
    OFP_VERSIONS = [ofproto_v1_3.OFP_VERSION]

    def __init__(self, *args, **kwargs):
        super(LearningSwitch, self).__init__(*args, **kwargs)
        self.switch_forwarding_table = {}

    @set_ev_cls(ofp_event.EventOFPSwitchFeatures, CONFIG_DISPATCHER)
    def switch_features_handler(self, ev):
        
        datapath = ev.msg.datapath
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser

        # Initial flow entry for matching misses
        match = parser.OFPMatch()
        actions = [parser.OFPActionOutput(ofproto.OFPP_CONTROLLER,
                                          ofproto.OFPCML_NO_BUFFER)]
        self.add_flow(datapath, 0, match, actions)

    def add_flow(self, datapath, priority, match, actions):
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser

        inst = [parser.OFPInstructionActions(ofproto.OFPIT_APPLY_ACTIONS, actions)]
        mod = parser.OFPFlowMod(datapath=datapath, priority=priority,
                                match=match, instructions=inst)
        datapath.send_msg(mod)
    



The above defines a RyuApp class with some basic boilerplate for configuring the controller. This remains mostly the same and can be reused.

The function `add_flow` adds the match-action to a flow table of a switch. Remember flow tables are unique to every switches.


```python
def add_flow(self, datapath, priority, match, actions):
        ofproto = datapath.ofproto
        parser = datapath.ofproto_parser

        inst = [parser.OFPInstructionActions(ofproto.OFPIT_APPLY_ACTIONS, actions)]
        mod = parser.OFPFlowMod(datapath=datapath, priority=priority,
                                match=match, instructions=inst)
        datapath.send_msg(mod)
```


Ryu works like a typical web server controller. It exposes certain events like EventSwitchEnter, EventSwitchLeave, EventOFPacketIn which can be handled by the user. A full list of events is provided here: https://github.com/faucetsdn/ryu/blob/master/ryu/topology/event.py and https://ryu.readthedocs.io/en/latest/ryu_app_api.html

* In this example, let's use the EventOFPacketIn event and write a handler to handle packets when they arrive.

In [ ]:
@set_ev_cls(ofp_event.EventOFPPacketIn, MAIN_DISPATCHER)
def _packet_in_handler(self, ev):
    msg = ev.msg
    datapath = msg.datapath

    in_port = msg.match['in_port']

    parser = datapath.ofproto_parser
    actions = [parser.OFPActionOutput(in_port)]

    out = parser.OFPPacketOut(
            datapath=datapath, buffer_id=msg.buffer_id, in_port=in_port,
            actions=actions, data=msg.data)
    
    datapath.send_msg(out)

L2SwitchController._packet_in_handler = _packet_in_handler

The above function is invoked everytime a packet is received by the controller. We then obtain details we need from the packet for further processing. In fact the most simplest handler you can write is the above:

The above handler will reroute every packet back to its own source. Not very useful, eh?



#### Key points:
* datapath is the switch which sent the packet to the controller.
* action is a list of actions that are executed in the order they are defined. There are several actions you can perform on a packet before sending it to a port. More details here: https://ryu.readthedocs.io/en/latest/ofproto_v1_3_ref.html#action-structures
* Openflow defines several fields that can also be matched. A full list is here: https://ryu.readthedocs.io/en/latest/ofproto_v1_3_ref.html

Although, the above definition works, its not so useful in its current state and we need to add more functionality to make it a L2 switch controller. Specifically, we need to do the following:

* Learn the in_port and mac_address combination whenever a packet comes to the controller.
* Add it to the flow table.
* If there is no prior knowledge, we flood the packet to every port.

Adding to flow table is as simple as invoking the add_flow method with match and action:

```python
actions = [parser.OFPActionOutput(out_port)]
match = parser.OFPMatch(eth_dst = eth.dst)
self.add_flow(datapath, of_proto.OFP_DEFAULT_PRIORITY, match, actions)
```

### Key points:
* Here we define a simple match-action pair where we say, if the packet matches the `eth.dst`, send it to the specific `out_port`.
* Why only those 2 values? It's entirely upto the developer to define match-action pair to match on anything and act in any way the developer wishes


### Here is a completed mac frame handler:

In [ ]:
@set_ev_cls(ofp_event.EventOFPPacketIn, MAIN_DISPATCHER)
def _packet_in_handler(self, ev):
    msg = ev.msg
    datapath = msg.datapath

    in_port = msg.match['in_port']

    parser = datapath.ofproto_parser
    of_proto = datapath.ofproto

    pkt = packet.Packet(msg.data)
    eth_pkt = pkt.get_protocol(ethernet.ethernet)

    if datapath.id not in self.switch_forwarding_table:
        self.switch_forwarding_table[datapath.id] = {}
    
    self.switch_forwarding_table[datapath.id][eth_pkt.src] = in_port
    actions = [parser.OFPActionOutput(of_proto.OFPP_FLOOD)]

    if eth_pkt.dst in self.switch_forwarding_table[datapath.id]:
        out_port = self.switch_forwarding_table[datapath.id][eth_pkt.dst]
        actions = [parser.OFPActionOutput(out_port)]
        match = parser.OFPMatch(eth_dst = eth_pkt.dst)

        self.logger.info("Added mac to flow table")
        self.add_flow(datapath, of_proto.OFP_DEFAULT_PRIORITY, match, actions)

    data = None

    if msg.buffer_id == of_proto.OFP_NO_BUFFER:
        data = msg.data
    
    if data is None:
        return
    
    out = parser.OFPPacketOut(
        datapath=datapath, buffer_id=msg.buffer_id, in_port=in_port,
        actions=actions, data=data)

    datapath.send_msg(out)


* As you can see, the logic is same as explained above.
* We check whether we know where to route eth.dst packet to. If not we flood the packet; otherwise we simply add a match-action entry to the flow table of that switch.
* Then we process the data and create an output response back to the switch.

### Key points:
* Why do we process the data this way?
* The data is only present if there is no buffer_id. Otherwise, it is parked in the switch and only the header content is sent to the controller.
* Here we process only packets if there is no buffer. (This is also upto the developer)
* We also obtain the required packet (ethernet, ipv4, arp, etc) using the packet parsing utilities provided by Ryu. More information here: https://ryu.readthedocs.io/en/latest/library_packet_ref.html

#### And that's it! This is the entire logic needed to implement a L2 networking switch in OpenFlow SDN using Ryu.

* The python version is in controllers/l2_switch_controller.py
* Continue reading the l3_switch_controller notebook to learn more about the next steps